In [314]:
import warnings
import random as rd
import simpy
import pandas as pd
from datetime import datetime
import statistics as st

warnings.filterwarnings("ignore")

In [315]:
df = pd.read_csv("simulated_call_centre.csv")

In [316]:
def convert_to_seconds(time_string):
    time_obj = datetime.strptime(time_string, "%I:%M:%S %p")
    time_in_seconds = (time_obj.hour * 3600) + (time_obj.minute * 60) + time_obj.second
    return time_in_seconds

def subtract(time_seconds):
    return time_seconds - 28800

def standard(wait_time):
    return int(wait_time) <= 60

def percentage(number1, number2):
    percentage1 = (number1 / (number1 + number2)) * 100
    percentage2 = (number2 / (number1 + number2)) * 100
    return percentage1, percentage2

In [317]:
# recorte da base (pior dia)
day_string = df.groupby('date')['wait_length'].sum().idxmax()
df_day = df[df["date"] == day_string]

# Tempo em String para dados numéricos
df_day['call_started'] = df_day['call_started'].apply(convert_to_seconds)
df_day['call_started'] = df_day['call_started'].apply(subtract)
df_day = df_day.drop(columns=["date","daily_caller","call_answered","call_ended"])

In [319]:
# remoção de outliers nos dados de entrada da simulação
start = df_day['call_started'].tolist()
lenght = df_day['service_length'].tolist()

start_length = zip(start,lenght)
start_length = [(i,j) for i,j in start_length if j != 1554 and j != 2244]

## Simulação Inicial

In [320]:
#variáveis da simulação
NUM_ATENDENTES = 4
SIM_TIME = 99999

In [321]:
class CallCentre:

    def __init__(self,env,num_atendentes):
        self.env = env
        self.atendente = simpy.Resource(env,num_atendentes)
        self.espera = []

    def atendimento(self,execution_time): # executa o tempo de atendimento 
        yield self.env.timeout(execution_time)

def cliente(env,nome,execution_time,cc): # gera um cliente

    with cc.atendente.request() as request:
        atendimento_solicitado = env.now
        print(f"Cliente {nome} solicita atendimento aos {atendimento_solicitado} segundos") # entra na simulação
        
        yield request
        atendimento_iniciado = env.now
        cc.espera.append(str(atendimento_iniciado - atendimento_solicitado))
        print(f"Cliente {nome} inicia atendimento aos {atendimento_iniciado} segundos") # é atendido

        yield env.process(cc.atendimento(execution_time))
        print(f"Cliente {nome} desliga aos {env.now} segundos") # sai da simulação

def run(env,cc): # executa a simulação
    nome = 0
    time_var = 0
    for start_time,execution_time in start_length:
        
        yield env.timeout(start_time-time_var)
        time_var = start_time
        nome += 1
        env.process(cliente(env,nome,execution_time,cc))

In [322]:
env = simpy.Environment()
call_centre = CallCentre(env,NUM_ATENDENTES)

env.process(run(env,call_centre))

print("saída:")
env.run(until=SIM_TIME)

result = pd.DataFrame(columns=['sim_wait_time','sim_meet_standard'])

result['sim_wait_time'] = call_centre.espera
result['sim_meet_standard'] = result['sim_wait_time'].apply(standard)

prob = result['sim_meet_standard'].value_counts()

True_values = percentage(prob[True],prob[False])
print(f"Porcentagem atendeu padrão: {True_values[0]}\nPorcentagem não atendeu padrão {True_values[1]}")

saída:
Cliente 1 solicita atendimento aos 0 segundos
Cliente 1 inicia atendimento aos 0 segundos
Cliente 2 solicita atendimento aos 139 segundos
Cliente 2 inicia atendimento aos 139 segundos
Cliente 3 solicita atendimento aos 174 segundos
Cliente 3 inicia atendimento aos 174 segundos
Cliente 2 desliga aos 392 segundos
Cliente 4 solicita atendimento aos 398 segundos
Cliente 4 inicia atendimento aos 398 segundos
Cliente 4 desliga aos 466 segundos
Cliente 5 solicita atendimento aos 560 segundos
Cliente 5 inicia atendimento aos 560 segundos
Cliente 1 desliga aos 579 segundos
Cliente 6 solicita atendimento aos 587 segundos
Cliente 6 inicia atendimento aos 587 segundos
Cliente 5 desliga aos 669 segundos
Cliente 6 desliga aos 704 segundos
Cliente 3 desliga aos 867 segundos
Cliente 7 solicita atendimento aos 1308 segundos
Cliente 7 inicia atendimento aos 1308 segundos
Cliente 8 solicita atendimento aos 1366 segundos
Cliente 8 inicia atendimento aos 1366 segundos
Cliente 9 solicita atendimento 

In [323]:
env = simpy.Environment()
call_centre = CallCentre(env,NUM_ATENDENTES+1) # aumento do número de atendentes

espera = env.process(run(env,call_centre))

print("saída:")
env.run(until=SIM_TIME)

result = pd.DataFrame(columns=['sim_wait_time','sim_meet_standard'])

result['sim_wait_time'] = call_centre.espera
result['sim_meet_standard'] = result['sim_wait_time'].apply(standard)

prob = result['sim_meet_standard'].value_counts()

True_values = percentage(prob[True],prob[False])
print(f"Porcentagem de padrão: {True_values[0]}\nPorcentagem não atendida {True_values[1]}")

saída:
Cliente 1 solicita atendimento aos 0 segundos
Cliente 1 inicia atendimento aos 0 segundos
Cliente 2 solicita atendimento aos 139 segundos
Cliente 2 inicia atendimento aos 139 segundos
Cliente 3 solicita atendimento aos 174 segundos
Cliente 3 inicia atendimento aos 174 segundos
Cliente 2 desliga aos 392 segundos
Cliente 4 solicita atendimento aos 398 segundos
Cliente 4 inicia atendimento aos 398 segundos
Cliente 4 desliga aos 466 segundos
Cliente 5 solicita atendimento aos 560 segundos
Cliente 5 inicia atendimento aos 560 segundos
Cliente 1 desliga aos 579 segundos
Cliente 6 solicita atendimento aos 587 segundos
Cliente 6 inicia atendimento aos 587 segundos
Cliente 5 desliga aos 669 segundos
Cliente 6 desliga aos 704 segundos
Cliente 3 desliga aos 867 segundos
Cliente 7 solicita atendimento aos 1308 segundos
Cliente 7 inicia atendimento aos 1308 segundos
Cliente 8 solicita atendimento aos 1366 segundos
Cliente 8 inicia atendimento aos 1366 segundos
Cliente 9 solicita atendimento 